# <center>Notebook para construção das tabelas do Reino Unido<center/>

Bases de Dados utilizadas:
* UK Gov Statistics

Bibliotecas

In [1]:
import pandas as pd
!pip install odfpy

In [2]:
data = pd.read_excel("../../data/external/2019_Table_1_New_STI_diagnoses_and_rates_in_England_by_gender.ods", engine="odf", sheet_name=3)
data

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,Table 1 (a): Number of new STI diagnoses in En...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,Data type: service data (where people access c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Published 02/09/2020,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,NaN,*level 1 chlamydia data only,Female,1944.64,1757.59,273.149,263.103,250.452,244.741505,236.069843,222.461303,229.458683,240.158431,-,10.6997
237,NaN,NaN,Total,1388.78,1236.13,205.897,195.564,183.71,183.522067,179.445393,165.834134,177.331197,189.603415,-,12.2722
238,NaN,Data for chlamydia & 'total new STI diagnoses'...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
239,NaN,Diagnosis totals from 2012 onwards include dat...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
filtered_data = pd.DataFrame(columns=['DST', 'Ano', 'Sexo', 'N-spec', 'spec', 'Total'])

In [4]:
dis_dict = {'Chlamydia - total': 'clamidia', 'Gonorrhoea - total': 'gonorreia', 'Herpes: anogenital herpes - total': 'herpes genital',
            'Mycoplasma genitalium': 'Mycoplasma genitalium', 'Non-specific genital infection - total': 'Não especificada',
            'Pelvic inflammatory disease (PID) & epididymitis:': 'DIP', 'Chlamydial PID & epididymitis - total': 'DIP (clamidia)',
            'Gonococcal PID & epididymitis - total': 'DIP (gonorreia)', 'Syphilis: primary, secondary & early latent - total': 'sifilis',
            'Warts: anogenital warts - total': 'verrugas genitais', 'Other new STI diagnoses - total': 'outras'}

In [5]:
# Total cases
for i in range(8, 106, 9):
    disease = data.iloc[i][1].strip().replace("**", "")
    for j in range(2):
        for k in range(5,13):
            ano = data.iloc[7][k]
            filtered_data = filtered_data.append({'DST': dis_dict[disease], 'Ano': int(ano), 'Sexo': 'M' if j == 0 else 'F',
                                  'N-spec': int(data.iloc[i+6+j][k]) if data.iloc[i+6+j][k] != '-' else '-',
                                  'spec': int(data.iloc[i+3+j][k]) if data.iloc[i+3+j][k] != '-' else '-',
                                 'Total': int(data.iloc[i+j][k]) if data.iloc[i+j][k] != '-' else '-'}, ignore_index=True)
filtered_data.to_csv('../../data/processed/uk_sti_cases.csv', index=False)

In [6]:
#Rates per 100000
filtered_data = pd.DataFrame(columns=['DST', 'Ano', 'Sexo', 'N-spec', 'spec', 'Total'])
for i in range(129, 228, 9):
    disease = data.iloc[i][1].strip().replace("**", "")
    for j in range(2):
        for k in range(5,13):
            ano = data.iloc[7][k]
            filtered_data = filtered_data.append({'DST': dis_dict[disease], 'Ano': int(ano), 'Sexo': 'M' if j == 0 else 'F',
                                  'N-spec': float(data.iloc[i+6+j][k]) if data.iloc[i+6+j][k] != '-' else '-',
                                  'spec': float(data.iloc[i+3+j][k]) if data.iloc[i+3+j][k] != '-' else '-',
                                 'Total': float(data.iloc[i+j][k]) if data.iloc[i+j][k] != '-' else '-'}, ignore_index=True)
filtered_data.to_csv('../../data/processed/uk_sti_cases_per_100000.csv', index=False)

In [7]:
dict_agent = {'bacteria': ['clamidia', 'gonorreia', 'Mycoplasma genitalium', 'DIP (clamidia)', 'DIP (gonorreia)', 'sifilis'], 
              'virus': ['herpes genital', 'verrugas genitais'],
               'N/A': ['Não especificada', 'DIP', 'outras']}
agent = pd.DataFrame(columns=['DST', 'agent'])
for k,v in dict_agent.items():
    for dis in v:
        agent = agent.append({'DST': dis, 'agent': k}, ignore_index=True)
agent.to_csv('../../data/processed/agente_causador.csv', index=False)